# Ridership table

This notebook creates a new ridership table.

Data is based on Bart's ridership data published here: https://www.bart.gov/about/reports/ridership
The table has 4 columns:

* *station_code*: 2 letter station code
* *exits*: average weekday exits from the station
* *normalized_exits*: exits for this station divided by the total number of exits
* *station_name*: name of the station that matches our tables from the excercise

In [130]:
import openpyxl

import psycopg2

import json

import csv

import openpyxl

import pandas as pd
import math
import numpy as np

In [131]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [132]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [133]:
cursor = connection.cursor()

In [134]:
def my_export_excel_work_book_sheet_to_csv(excel_file_name, sheet_name, csv_file_name):
    "open the excel file, find the sheet_name, export to csv file "
    
    work_book = openpyxl.load_workbook(excel_file_name)
    
    work_sheet = work_book[sheet_name]
                                       
    print("\nExcel Workbook:", excel_file_name)
          
    print("\nWork Sheet:", sheet_name)
    
    print("\nNumber of Columns:", work_sheet.max_column)
          
    print("\nNumber of Rows:", work_sheet.max_row)
    
    work_sheet_list = []
    
    for i in range(1,work_sheet.max_row+1):
        
        row_list = []
        
        for j in range(1,work_sheet.max_column+1):
            
            row_list.append(work_sheet.cell(row = i, column = j).value)
            
        work_sheet_list.append(row_list)
    
    print("\nExtracting Work Sheet", sheet_name, "to", csv_file_name, "\n")
    
    f = open(csv_file_name, "w")    
    writer = csv.writer(f)
    writer.writerows(work_sheet_list)
    f.close()

In [135]:
my_export_excel_work_book_sheet_to_csv("Ridership_202410.xlsx", "Average Weekday", "ridership_202410_average_weekday.csv")


Excel Workbook: Ridership_202410.xlsx

Work Sheet: Average Weekday

Number of Columns: 52

Number of Rows: 55

Extracting Work Sheet Average Weekday to ridership_202410_average_weekday.csv 



# Drop ridership table if it exists

In [147]:
connection.rollback()

query = """

DROP TABLE IF EXISTS ridership;

"""

cursor.execute(query)

connection.commit()


# Create the ridership table

In [148]:
connection.rollback()

query = """

CREATE TABLE ridership (
    station_code varchar(2) PRIMARY KEY,
    station_name varchar(32),
    exits numeric(9),
    normalized_exits DOUBLE PRECISION
);

"""

cursor.execute(query)

connection.commit()

# Load ridership CSV into table

In [149]:
# Skip the first 4 rows which have headers that we can't really parse
df = pd.read_csv('ridership_202410_average_weekday.csv', skiprows=3)
# Skip the last row which has totals for all stations.
df_filtered = df[:-1]
# First column has station codes and last column has average station exits
df_filtered = df_filtered.iloc[:, [0, -1]]

In [150]:
def insert_ridership_row(station_code, exits, normalized_exits):
    connection.rollback()

    query = f"""
        INSERT INTO ridership
            (station_code, exits, normalized_exits)
        VALUES
            ('{station_code}', {exits}, {normalized_exits})
        """

    cursor.execute(query)

    connection.commit()

In [151]:
total_exits = 0
for _, row in df_filtered.iterrows():
    exits = row[1]
    total_exits += exits

for _, row in df_filtered.iterrows():
    station_code = row[0]
    exits = row[1]
    normalized_exits = exits / total_exits
    insert_ridership_row(station_code, exits, normalized_exits)

# Map station code to station name

In [152]:
my_export_excel_work_book_sheet_to_csv("station-names.xlsx", "DAS Station Codes", "station-names.csv")


Excel Workbook: station-names.xlsx

Work Sheet: DAS Station Codes

Number of Columns: 7

Number of Rows: 51

Extracting Work Sheet DAS Station Codes to station-names.csv 



In [153]:
df = pd.read_csv('station-names.csv')

In [154]:
def update_station_name_row(station_code, station_name):
    connection.rollback()
    
    query = f"""
        UPDATE ridership
        SET station_name = '{station_name}'
        WHERE station_code = '{station_code}'
    """

    cursor.execute(query)

    connection.commit()

In [155]:
for _, row in df.iterrows():
    station_code = row[1]
    station_name = row[3]
    update_station_name_row(station_code, station_name)

# Display the ridership table

In [156]:
rollback_before_flag = True
rollback_after_flag = True

query = f"""
    SELECT *
    FROM ridership
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station_code,station_name,exits,normalized_exits
0,RM,Richmond,2456,0.013580
1,EN,El Cerrito del Norte,4032,0.022297
2,EP,El Cerrito Plaza,2186,0.012089
3,NB,North Berkeley,1820,0.010064
4,BK,Downtown Berkeley,6172,0.034133
5,AS,Ashby,2077,0.011487
6,MA,MacArthur,3924,0.021698
7,19,19th Street,4891,0.027048
8,12,12th Street,5243,0.028993
9,LM,Lake Merritt,3063,0.016936


# Ridership visualization

In [124]:
import json

import gmaps
import gmaps.geojson_geometries

from geographiclib.geodesic import Geodesic

In [125]:
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

In [126]:
sather_gate_berkeley = (37.870260430419115, -122.25950168579497)

In [159]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT station, latitude, longitude
FROM stations
ORDER BY station

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

In [128]:
def get_exits_for_station(station):
    query = f"""

    SELECT exits
    FROM ridership
    WHERE station_name = '{station}'
    ORDER by 1

    """

    cursor.execute(query)

    connection.rollback()

    rows = cursor.fetchall()
    if not rows:
        print(station)
    return int(rows[0][0])

In [165]:
def get_normalized_exits_for_station(station):
    query = f"""

    SELECT normalized_exits
    FROM ridership
    WHERE station_name = '{station}'
    ORDER by 1

    """

    cursor.execute(query)

    connection.rollback()

    rows = cursor.fetchall()
    if not rows:
        print(station)
    return rows[0][0]

In [178]:
fig = gmaps.figure(center=sather_gate_berkeley, zoom_level=9)

circles = []

for _, row in df.iterrows():
    station = row[0]
    latitude = row[1]
    longitude = row[2]
    normalized_exits = get_normalized_exits_for_station(station)
    if not normalized_exits:
        print(row)
        continue
    circles.append(gmaps.Circle(
        center=(latitude, longitude),
        radius=30000 * normalized_exits,
        fill_color = "red",
        fill_opacity = 0.5,
        stroke_color = "red",
        stroke_weight = 1
    ))
    

circle_layer = gmaps.drawing_layer(features=circles)

fig.add_layer(circle_layer)

fig

Figure(layout=FigureLayout(height='420px'))